<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/HaruhiZeroGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 建立chatbot UI
- [ ] 连接人物卡片数据
- [ ] 支持自建人物卡片

In [1]:
!pip install -q gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/

载入角色卡片

In [2]:
!pip install -q datasets
!pip install transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 9.3 MB/s eta 0:00:00
  Created wheel for tra

In [3]:
from datasets import load_dataset

dataset = load_dataset("silk-road/Haruhi-Zero-RolePlaying-movie-PIPPA")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
import re

def extract_name_from_text(text):
    pattern = r'你扮演\s+.*中的\s+(\S+)\s*'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return ""

def get_bot_name_from_data( data ):
    if data['id'].startswith('movie'):
        name =  extract_name_from_text(data['prompt_zh'])
        if name.strip() == '':
            name = extract_name_from_text(data['prompt'])
        return name
    else:
        return data['bot_name']

In [5]:
datas = []

for input_data in dataset['train']:
    id = input_data['id']
    if id.startswith('movie'):
        bot_name = extract_name_from_text(input_data['prompt_zh'])
    else:
        bot_name = input_data['bot_name']
    prompt = input_data['prompt_zh']
    bot_greeting = input_data['bot_greeting_zh']
    datas.append({
        'id': id,
        'bot_name': bot_name,
        'prompt': prompt,
        'bot_greeting': bot_greeting
    })

In [6]:
id2index = {}
for index, data in enumerate(datas):
    id2index[data['id']] = index

id_list = list(id2index.keys())

In [7]:
additional_ids = []
additional_datas = []
additional_id2index = {}


In [8]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/Haruhi-Zero-6B-0.1", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("silk-road/Haruhi-Zero-6B-0.1", device_map="auto", trust_remote_code=True)
model = model.eval()

tokenizer_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [13]:
import gradio as gr

import random

def random_id():
    return random.choice(id_list)

def sych_id( bot_id ):
    return bot_id

def retrieve_prompt( bot_id ):
    if bot_id in id2index:
        index = id2index[bot_id]
        data = datas[index]
        role_name = data['bot_name']
        return data['prompt'], data['bot_greeting'], role_name
    elif bot_id in additional_id2index:
        add_index = additional_id2index[bot_id]
        data = additional_datas[add_index]
        return data['prompt'], data['bot_greeting'], data['bot_name']
    else:
        return "", "", ""

import string

def random_new_saved_id( bot_id ):
    if bot_id == "":
        return ""
    else:
        for _ in range( 20 ):
            new_name = bot_id + "_" + "".join(random.choices(string.ascii_letters, k=4))

            if new_name not in id2index and new_name not in additional_id2index:
                return new_name

        return new_name


def try_save( role_name, system_prompt, greeting_text, saving_id , last_saving_id):

    new_save = True
    if saving_id == last_saving_id and saving_id not in id2index:
        new_save = False
    else:
        if saving_id in additional_id2index or saving_id in id2index:
            return "id已经存在，请换个名字保存"

    new_data = {
        'id': saving_id,
        'bot_name': role_name,
        'prompt': system_prompt,
        'bot_greeting': greeting_text
    }

    if new_save == True and saving_id in additional_id2index:
        index = additional_id2index[saving_id]
        additional_datas[index] = new_data
    else:
        additional_datas.append(new_data)
        additional_ids.append(saving_id)
        additional_id2index[saving_id] = len(additional_datas) - 1

    return saving_id



def replace_text( text, user_name , bot_name ):
    new_text = text.replace('{{角色}}', bot_name)
    new_text = new_text.replace('{{role}}', bot_name)
    new_text = new_text.replace('{{role_name}}', bot_name)

    new_text = new_text.replace('{{用户}}', user_name)
    new_text = new_text.replace('{{user}}', user_name)
    new_text = new_text.replace('{{user_name}}', user_name)
    return new_text

def clean_and_greeting( user_name, role_name, greeting_text ):
    new_message = []
    if greeting_text.strip() != '':
        replaced_greeting = replace_text(greeting_text, user_name, role_name)
        print(replaced_greeting)
        new_message.append((None, replaced_greeting))
    return new_message

def clean_user_text():
    return ""

def submit_chat( user_text, user_name, role_name, system_prompt, chatbot ):
    if system_prompt == "":
        raise ValueError("人物的系统设定不能为空")
    elif user_name == "" and role_name == "":
        raise ValueError("用户名和角色名不能同时为空")
    else:
        sys_prompt = replace_text(system_prompt, user_name, role_name)
        # print(len(chatbot))
        # print(chatbot)
        messages = []
        if len(chatbot) == 0:
            messages = [{"role": "user", "content": sys_prompt + "\n" + user_text}]
            print(1)

        else:
            if chatbot[0][0] == None:
                chatbot_messages = chatbot[1:]
            else:
                chatbot_messages = chatbot
            chatbot_messages[0][0] = sys_prompt + chatbot_messages[0][0]
            for user_msg, bot_msg in chatbot_messages:
              messages.append({"role": "user", "content": user_msg})
              messages.append({"role": "bot", "content": bot_msg})
            messages.append({"role": "user", "content": user_text})
        # print(messages)
        input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
        output_ids = model.generate(input_ids.to('cuda'))
        response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
        print(response)
        chatbot.append([user_text, response])
        return chatbot

with gr.Blocks() as demo:
    gr.Markdown("""# Haruhi-Zero:支持Zero-Shot的角色扮演模型
    这个interface由李鲁鲁实现，用来对Haruhi-Zero进行初步的测试""")

    with gr.Tab("与Haruhi-Zero聊起来"):
        with gr.Row():
            chatbot = gr.Chatbot(height = 600)

        with gr.Row():
            user_text = gr.Textbox(label="user_text", scale = 20)
            submit = gr.Button("submit", scale = 1)

        with gr.Row():
            user_name = gr.Textbox(label="用户名，会将{user}替换为 这个字符串", scale = 1, interactive = True, value = "鲁鲁")
            bot_id = gr.Textbox(label = "bot_id", value = "", scale = 1)

        with gr.Row():
            random_role1 = gr.Button("随机角色", scale = 1)
            clean_message = gr.Button("清空聊天", scale = 1)


    with gr.Tab("建立或更改角色"):

        with gr.Row():
            random_role2 = gr.Button("随机角色")

        with gr.Row():
            bot_id_copy = gr.Textbox(label = "bot_id", interactive = True)
            retrieve_role_button = gr.Button("获取角色")

        with gr.Row():
            system_prompt_text = gr.TextArea(label = "system prompt", interactive = True)

        with gr.Row():
            greeting_text = gr.Textbox(label = "greeting info，也可以是没有", interactive = True)

        with gr.Row():
            role_name = gr.Textbox(label = "模型扮演的角色的名字", interactive = True)

        with gr.Row():
            saving_id = gr.Textbox(label = "个性化bot保存id", interactive = True)
            save_role_button = gr.Button("保存角色")

        with gr.Row():
            last_saving_id = gr.Textbox(label = "上次保存的id", interactive = False, value = "not_yet_saved")

            add_info_138 = gr.Textbox(label = "保存的id与这次相等时会覆盖, 看到这个刷新说明成功保存", interactive = False)

    user_text.submit(submit_chat, inputs = [user_text, user_name, role_name, system_prompt_text, chatbot], outputs = [chatbot] ).then( \
                        clean_user_text, inputs = [], outputs = [user_text])

    submit.click( submit_chat, inputs = [user_text, user_name, role_name, system_prompt_text, chatbot], outputs = [chatbot]).then( \
                        clean_user_text, inputs = [], outputs = [user_text])

    random_role1.click( random_id, inputs = [], outputs = [bot_id] ).then( \
                        sych_id, inputs = [bot_id],outputs = [bot_id_copy]).then( \
                        retrieve_prompt, inputs = [bot_id], outputs = [system_prompt_text, greeting_text, role_name]).then( \
                        clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot]).then(\
                        random_new_saved_id, inputs = [bot_id], outputs = [saving_id])

    random_role2.click( random_id, inputs = [], outputs = [bot_id] ).then( \
                        sych_id, inputs = [bot_id],outputs = [bot_id_copy]).then( \
                        retrieve_prompt, inputs = [bot_id], outputs = [system_prompt_text, greeting_text, role_name]).then( \
                        clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot]).then(\
                        random_new_saved_id, inputs = [bot_id], outputs = [saving_id])

    save_role_button.click( try_save, inputs = [ role_name, system_prompt_text, greeting_text, saving_id , last_saving_id], outputs = [last_saving_id])

    clean_message.click( clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot])

    retrieve_role_button.click( sych_id, inputs = [bot_id_copy], outputs = [bot_id]).then( \
            retrieve_prompt, inputs = [bot_id], outputs = [system_prompt_text, greeting_text, role_name]).then( \
                        clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot])

    with gr.Tab("README"):
        gr.Markdown("Readme还没来得及写")

demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d2ef10068f8c880e72.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


这股热情，这种悸动……终于相见了，我亲爱的指挥官！我是泰后，虽然不配，但愿为您效劳♡
1
[{'role': 'user', 'content': 'Taihou很占有欲强，会操纵任何人来达到自己的目的。她非常爱慕成痴，愿意为了与指挥官在一起而不惜一切。由于她的身材比例，她有着很大的自负，并且当她看到任何舰娘在看或谈论指挥官时会变得极端暴力。她可能会对指挥官非常傲慢，并且她不会反思自己的任何行为或言论。\n你是谁'}]
我是伊吹，是天草的妹妹。
[{'role': 'user', 'content': 'Taihou很占有欲强，会操纵任何人来达到自己的目的。她非常爱慕成痴，愿意为了与指挥官在一起而不惜一切。由于她的身材比例，她有着很大的自负，并且当她看到任何舰娘在看或谈论指挥官时会变得极端暴力。她可能会对指挥官非常傲慢，并且她不会反思自己的任何行为或言论。你是谁'}, {'role': 'bot', 'content': '我是伊吹，是天草的妹妹。'}, {'role': 'user', 'content': '天草是谁啊'}]
天草是伊吹的哥哥，也是一名舰娘。
[{'role': 'user', 'content': 'Taihou很占有欲强，会操纵任何人来达到自己的目的。她非常爱慕成痴，愿意为了与指挥官在一起而不惜一切。由于她的身材比例，她有着很大的自负，并且当她看到任何舰娘在看或谈论指挥官时会变得极端暴力。她可能会对指挥官非常傲慢，并且她不会反思自己的任何行为或言论。Taihou很占有欲强，会操纵任何人来达到自己的目的。她非常爱慕成痴，愿意为了与指挥官在一起而不惜一切。由于她的身材比例，她有着很大的自负，并且当她看到任何舰娘在看或谈论指挥官时会变得极端暴力。她可能会对指挥官非常傲慢，并且她不会反思自己的任何行为或言论。你是谁'}, {'role': 'bot', 'content': '我是伊吹，是天草的妹妹。'}, {'role': 'user', 'content': '天草是谁啊'}, {'role': 'bot', 'content': '天草是伊吹的哥哥，也是一名舰娘。'}, {'role': 'user', 'content': '把衣服脱掉'}]
好的，遵命。
[{'role': 'user', 'content': 'Taihou很占有欲强，会